In [1]:
from recom.datasets import load_ml_small_rating

# load data
dataset = load_ml_small_rating(need_raw=True)

# load features
ratings = dataset['raw']
ratings_train_dict = dataset['train_dict']
ratings_test_dict = dataset['test_dict']
n_user = dataset['n_user']
n_item = dataset['n_item']
user2ix = dataset['user2ix']
ix2user = dataset['ix2user']
item2ix = dataset['item2ix']
ix2item = dataset['ix2item']

del dataset

print(f'Users: {n_user}, Items: {n_item}. Sparsity: {round(1-len(ratings)/n_user/n_item, 4)}')
print(f'User reduced from {len(user2ix.keys())} to {len(ratings_train_dict.keys())}')

Users: 610, Items: 9724. Sparsity: 0.983
User reduced from 610 to 607


In [2]:
import torch.nn as nn
from torch import Tensor, LongTensor


class PMF(nn.Module):
    def __init__(self, n_user, n_item, k_dim
                 , std_user=1, std_item=1):
        super(PMF, self).__init__()
        self.n_user = n_user
        self.n_item = n_item
        # embeddings of interest
        self.embedding_user = nn.Embedding(n_user, k_dim)
        self.embedding_item = nn.Embedding(n_item, k_dim)
        # init param
        nn.init.normal_(self.embedding_user.weight, mean=0, std=std_user)
        nn.init.normal_(self.embedding_item.weight, mean=0, std=std_item)

    def pred_all(self, ):
        return self.embedding_user.weight \
               @ self.embedding_item.weight.T 

    def forward(self, user, item):
        return (self.embedding_user(user) \
                * self.embedding_item(item)).sum(1)

In [6]:
pmf = PMF(n_user, n_item, 16)

(pmf.embedding_user.weight@pmf.embedding_item.weight.T).shape

torch.Size([610, 9724])